In [1]:
%run read_file.ipynb

Matplotlib is building the font cache; this may take a moment.


:: loading settings :: url = jar:file:/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ec2-user/.ivy2/cache
The jars for the packages stored in: /home/ec2-user/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-9a51c2eb-f440-4cab-b356-a760c19697eb;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
downloading https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar ...
	[SUCCESSFUL ] graphframes#graphframes;0.8.2-spark3.2-s_2.12!graphframes.jar (51ms)
downloading https://repo1.maven.org/maven2/org/slf4j/slf4j-api/1.7.16/slf4j-api-1.7.16.jar ...
	[SUCCESSFUL ] org.slf4j#slf4j-api;1.7.16!slf4j-api.jar (39ms)
:: resolution report :: resolve 834ms :: artifacts dl 93ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	--------

23/04/01 12:47:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/01 12:47:11 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [ ]:
%run trace_encode.ipynb

In [ ]:
df_all = de_dupe()

In [ ]:
df_all = df_all.cache()

In [ ]:
df_all.limit(2).toPandas()

In [ ]:
df_all.printSchema()

In [ ]:
tot = df_all.count()

In [ ]:
#check days and malicious counts
df_all.groupBy("event_day", "malicious") \
              .count().cache().show()

In [ ]:
df_all = df_all.cache()

In [ ]:
#how many image paths are there
df_all.groupBy('image_path_ext','parent_path_ext').count().show()

In [ ]:
list_img = ['None','empty','.exe','.com']
list_par_img = ['None','empty','.exe']

In [ ]:
#how many new paths are there
df_all.select('new_path_ext').distinct().count()

In [ ]:
#what are the new path counts
df_all.groupBy('new_path_ext').count().show()

In [ ]:
#how many file paths are there
df_all.select('file_path_ext').distinct().count()

In [ ]:
#how many file paths are there
ext_ratios = df_all.groupBy('file_path_ext','malicious').count().sort(col('count').desc()).withColumn('perc_of_count_total', (col('count') / tot) * 100).limit(20)


In [ ]:
ext_ratios.show()

In [ ]:
#create a list of top 20 file extensions
file_ext = list(df_all.groupBy('file_path_ext').count().sort(col('count').desc()).limit(9).toPandas()['file_path_ext'])

In [ ]:
df_all = df_all.withColumn("image_path", getFileUDF(col("image_path"))).cache()
df_all = df_all.withColumn("parent_image_path", getFileUDF(col("parent_image_path"))).cache()

In [ ]:
#how many file paths are there
ext_ratios_img = df_all.groupBy('image_path').count().sort(col('count').desc()).withColumn('perc_of_count_total', (col('count') / tot) * 100).limit(20)
ext_ratios_parent = df_all.groupBy('parent_image_path').count().sort(col('count').desc()).withColumn('perc_of_count_total', (col('count') / tot) * 100).limit(20)

In [ ]:
ext_ratios_img.show()
ext_ratios_parent.show()

In [ ]:
#create a list of top 20 file extensions
img_path = list(df_all.groupBy('image_path').count().sort(col('count').desc()).limit(16).toPandas()['image_path'])
#create a list of top 20 file extensions
par_path = list(df_all.groupBy('parent_image_path').count().sort(col('count').desc()).limit(6).toPandas()['parent_image_path'])

In [ ]:
print(img_path)
print(par_path)

In [ ]:
df_23 = df_all.where(col('event_day') == 23).cache()

In [ ]:
df_23_1516 = df_23.where((col('event_hour') == 16) | (col('event_hour') == 15)).cache()

In [ ]:
df_23_1516.count()

In [ ]:
df_23_15 = df_23.where((col('event_hour') == 15)).cache()

In [ ]:
df_23_15.count()

In [ ]:
#now create graph for event traces of 2 events, 4 events, and 6 events
start_time = time.time()
    
#create trace matrix from malicious events for speed. 
# Create distinct vertices with source as actorid, destination as objectid for malicious
src_vertices = df_23_15.selectExpr('actorID as id').distinct()
dst_vertices = df_23_15.selectExpr('objectID as id').distinct()
vertices = src_vertices.union(dst_vertices).distinct()

# Create edges by using timestamp as an edge
edges = df_23_15.selectExpr('objectID as src', 'actorID as dst', 'timestamp', 'object', 'action', 'hostname', 'user_name', 'privileges', 'image_path',
                          'parent_image_path', 'new_path', 'file_path', 'direction', 'logon_id', 'requesting_domain', 'requesting_user', 'malicious')

# Create GraphFrame
g = GraphFrame(vertices, edges)
motifs6 = g.find("(a)-[e1]->(b); (b)-[e2]->(c)") #; (c)-[e3]->(d); (d)-[e4]->(e); (e)-[e5]->(f); (f)-[e6]->(g)")
print("found connections: "+ str(time.time() - start_time))
#create paths and count
# filter paths to only those where all edges are connected
cp_2315_2 = motifs6.filter('''e1.timestamp <= e2.timestamp''').cache()# and e2.timestamp <= e3.timestamp and 
                                    #e3.timestamp <= e4.timestamp and e4.timestamp <= e5.timestamp and 
                                    #e5.timestamp <= e6.timestamp''').cache()

print("event traces: "+str(cp_2315_2.count()))

print("create graph: "+ str(time.time() - start_time))

In [ ]:
#now create graph for event traces of 2 events, 3 events
start_time = time.time()
    
#create trace matrix from malicious events for speed. 
# Create distinct vertices with source as actorid, destination as objectid for malicious
src_vertices = df_23_15.selectExpr('actorID as id').distinct()
dst_vertices = df_23_15.selectExpr('objectID as id').distinct()
vertices = src_vertices.union(dst_vertices).distinct()

# Create edges by using timestamp as an edge
edges = df_23_15.selectExpr('actorID as src', 'objectID as dst', 'timestamp', 'object', 'action', 'hostname', 'user_name', 'privileges', 'image_path',
                          'parent_image_path', 'new_path', 'file_path', 'direction', 'logon_id', 'requesting_domain', 'requesting_user', 'malicious')

# Create GraphFrame
g = GraphFrame(vertices, edges)
motifs6 = g.find("(a)-[e1]->(b); (b)-[e2]->(c); (c)-[e3]->(d)") #; (d)-[e4]->(e)") #; (e)-[e5]->(f); (f)-[e6]->(g)")
print("found connections: "+ str(time.time() - start_time))
#create paths and count
# filter paths to only those where all edges are connected
cp_2315_3 = motifs6.filter('''e1.timestamp <= e2.timestamp and e2.timestamp <= e3.timestamp''')# and 
                                    #e3.timestamp <= e4.timestamp''').cache()# and e4.timestamp <= e5.timestamp and 
                                    #e5.timestamp <= e6.timestamp''').cache()

print("event traces: "+str(cp_2315_3.count()))

print("create graph: "+ str(time.time() - start_time))

In [ ]:
#now create graph for event traces of 2 events, 3 events
start_time = time.time()
    
#create trace matrix from malicious events for speed. 
# Create distinct vertices with source as actorid, destination as objectid for malicious
src_vertices = df_23_15.selectExpr('actorID as id').distinct()
dst_vertices = df_23_15.selectExpr('objectID as id').distinct()
vertices = src_vertices.union(dst_vertices).distinct()

# Create edges by using timestamp as an edge
edges = df_23_15.selectExpr('actorID as src', 'objectID as dst', 'timestamp', 'object', 'action', 'hostname', 'user_name', 'privileges', 'image_path',
                          'parent_image_path', 'new_path', 'file_path', 'direction', 'logon_id', 'requesting_domain', 'requesting_user', 'malicious')

# Create GraphFrame
g = GraphFrame(vertices, edges)
motifs6 = g.find("(a)-[e1]->(b); (b)-[e2]->(c); (c)-[e3]->(d)") #; (d)-[e4]->(e)") #; (e)-[e5]->(f); (f)-[e6]->(g)")
print("found connections: "+ str(time.time() - start_time))
#create paths and count
# filter paths to only those where all edges are connected
cp_2315_3 = motifs6.filter('''e1.timestamp <= e2.timestamp and e2.timestamp <= e3.timestamp''')# and 
                                    #e3.timestamp <= e4.timestamp''').cache()# and e4.timestamp <= e5.timestamp and 
                                    #e5.timestamp <= e6.timestamp''').cache()

print("event traces: "+str(cp_2315_3.count()))

print("create graph: "+ str(time.time() - start_time))

In [ ]:
#now create graph for event traces of 2 events, 3 events
start_time = time.time()
    
#create trace matrix from malicious events for speed. 
# Create distinct vertices with source as actorid, destination as objectid for malicious
src_vertices = df_23_15.selectExpr('objectID as id').distinct()
dst_vertices = df_23_15.selectExpr('actorID as id').distinct()
vertices = src_vertices.union(dst_vertices).distinct()

# Create edges by using timestamp as an edge
edges = df_23_15.selectExpr('objectID as src', 'actorID as dst', 'timestamp', 'object', 'action', 'hostname', 'user_name', 'privileges', 'image_path',
                          'parent_image_path', 'new_path', 'file_path', 'direction', 'logon_id', 'requesting_domain', 'requesting_user', 'malicious')

# Create GraphFrame
g = GraphFrame(vertices, edges)
motifs6 = g.find("(a)-[e1]->(b); (b)-[e2]->(c); (c)-[e3]->(d)") #; (d)-[e4]->(e)") #; (e)-[e5]->(f); (f)-[e6]->(g)")
print("found connections: "+ str(time.time() - start_time))
#create paths and count
# filter paths to only those where all edges are connected
cp_2315_3oa = motifs6.filter('''e1.timestamp <= e2.timestamp and e2.timestamp <= e3.timestamp''')# and 
                                    #e3.timestamp <= e4.timestamp''').cache()# and e4.timestamp <= e5.timestamp and 
                                    #e5.timestamp <= e6.timestamp''').cache()

print("event traces: "+str(cp_2315_3oa.count()))

print("create graph: "+ str(time.time() - start_time))

In [ ]:
df_transp = cp_2315_3oa.withColumn("Trace", (monotonically_increasing_id() + 1))
df_transp = df_transp.select("Trace", 
                             *[col for col in df_transp.columns if col != "Trace"])

#drop all vertices
df_transp = df_transp.drop('a','b','c','d')

#transpose rows 
stacked_df = df_transp.selectExpr(
    "Trace", 
    "posexplode(array(e1, e2, e3)) as (pos, col)"
).select(
    "Trace", 
    expr('''CASE pos 
    WHEN 0 THEN 'e1' 
    WHEN 1 THEN 'e2'
    ELSE 'e3' END''').alias("event"),
    "col"
).orderBy("Trace","event")

#explode columns
df_onehot_oa = stacked_df.select(*stacked_df.columns, "col.*").drop('col').cache()

df_transp = cp_2315_3.withColumn("Trace", (monotonically_increasing_id() + 1))
df_transp = df_transp.select("Trace", 
                             *[col for col in df_transp.columns if col != "Trace"])

#drop all vertices
df_transp = df_transp.drop('a','b','c','d')

#transpose rows 
stacked_df = df_transp.selectExpr(
    "Trace", 
    "posexplode(array(e1, e2, e3)) as (pos, col)"
).select(
    "Trace", 
    expr('''CASE pos 
    WHEN 0 THEN 'e1' 
    WHEN 1 THEN 'e2'
    ELSE 'e3' END''').alias("event"),
    "col"
).orderBy("Trace","event")

#explode columns
df_onehot = stacked_df.select(*stacked_df.columns, "col.*").drop('col').cache()

In [ ]:
df_onehot_oa.limit(6).toPandas()

In [ ]:
df_onehot.limit(6).toPandas()

In [ ]:
#write both samples to parquet
s3_url_trusted = "s3a://sapient-bucket-trusted/"
cp_2315_2.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/test/2315_2")

In [ ]:
#write both samples to parquet
s3_url_trusted = "s3a://sapient-bucket-trusted/"
cp_2315_4.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/test/2315_4")

## So now let's work on filtering

In [ ]:
#create the necessary file extension filtering
df = df_onehot_oa.withColumn("image_path_ext", regexp_extract("image_path", "\.[0-9a-z]+$", 0)) \
        .withColumn("parent_path_ext", regexp_extract("parent_image_path", "\.[0-9a-z]+$", 0)) \
        .withColumn("new_path_ext", regexp_extract("new_path", "\.[0-9a-z]+$", 0)) \
        .withColumn("file_path_ext", regexp_extract("file_path", "\.[0-9a-z]+$", 0)).cache()

In [ ]:
tot = df.count()

In [ ]:
df.groupBy('file_path_ext').count().sort(col('count').desc()).withColumn('perc_of_count_total', (col('count') / tot) * 100).limit(20).show()

In [ ]:
file_ext = [str(i) for i in file_ext]

In [ ]:
file_ext = ['None', 'empty', '.dll', '.pyc', '.bat', '.pf', '.ini', '.exe', '.tmp']

In [ ]:
df = df.withColumn("file_path_ext", when(col("file_path_ext").isNull(),"None").otherwise(col("file_path_ext"))) \
.withColumn("file_path_ext", when(col("file_path_ext") == '', 'empty').otherwise(col("file_path_ext"))) \
.withColumn("file_path_ext", when(col("file_path_ext").isin(file_ext), col('file_path_ext')).otherwise('other')).cache()

In [ ]:
list_img = ['None','empty','.exe','.com']
list_par_img = ['None','empty','.exe']

df = df.withColumn("image_path_ext", when(col("image_path_ext").isNull(),"None").otherwise(col("image_path_ext"))) \
.withColumn("image_path_ext", when(col("image_path_ext") == '', 'empty').otherwise(col("image_path_ext"))) \
.withColumn("image_path_ext", when(col("image_path_ext").isin(list_img), col('image_path_ext')).otherwise('other')).cache()

In [ ]:
df = df.withColumn("parent_path_ext", when(col("parent_path_ext").isNull(),"None").otherwise(col("parent_path_ext"))) \
.withColumn("parent_path_ext", when(col("parent_path_ext") == '', 'empty').otherwise(col("parent_path_ext"))).cache()

In [ ]:
df.select('image_path_ext').distinct().show()

In [ ]:
df.select('parent_path_ext').distinct().show()

In [ ]:
#now filter the file paths based on the 9 we selected above
df = df.withColumn("file_path_ext",
                             when(col("file_path_ext").isin(file_ext), 
                                  col('file_path_ext')).otherwise('other')).cache()

In [ ]:
#check results
df.select('file_path_ext').distinct().show()

In [ ]:
df.select('image_path_ext').distinct().show()

In [ ]:
df.select('parent_path_ext').distinct().show()

In [ ]:
#now work on labeling entire malicious trace
# create a sample dataframe that aligns closer with what you see above. 
#I think the trick is going to be partitioning based on object action and timestamp.
#if those three things are the same then pull the first one out
data = [("A", "B",1, 1,'process','create',0), 
        ("B", "C",1, 2,'process','create',0), 
        ("C", "D",1, 3,'process','open',0),
        ("D", "E",2,4,'process','open',1),
        ("E", "F",2, 5,'file','read',0),
       ("F", "G",2, 6,'process','create',0), 
        ("G", "H",3, 7,'process','create',0), 
        ("H", "I",3, 8,'process','open',1),
        ("I", "J" ,3,9,'process','open',0),
        ("J", "K",3, 10,'file','read',1),
       ("K", "L",4, 8,'process','open',1),
        ("M", "N",4,9,'process','open',1),
        ("O", "P",4, 10,'file','read',1)]
columns = ["src", "dst","Trace", "time","object","action",'malicious']
df_samp = spark.createDataFrame(data, columns)

In [ ]:
#create a window method to label traces as malicious given one event
w = Window.partitionBy("Trace")#.orderBy("time")

# Apply the conditions to the window using the when function
df_samp = df_samp.withColumn('mal_trace', when(sum('malicious').over(w) > 0, 1).otherwise(0))
#df_samp = df_samp.withColumn('mal_trace', sum('mal_trace').over(w))

In [ ]:
df_samp.show()

In [ ]:
#calculate malicious trace then check schema and how many were made 
w = Window.partitionBy("Trace")
df = df.withColumn('mal_trace', when(sum('malicious').over(w) > 0, 1).otherwise(0)).cache()

In [ ]:
df.printSchema()

In [ ]:
df.groupby('mal_trace').count().show()

In [ ]:
for c in ['file_path_ext','image_path_ext','parent_path_ext','object','action']:
    df, dict_mapping, cols_sparse = oneHotCol(df, c, {}, [])

In [ ]:
df.printSchema()

In [ ]:
df.limit(2).toPandas()

In [ ]:
#calculate malicious trace then check schema and how many were made 
w = Window.partitionBy("Trace")
df = df.withColumn('mal_trace', when(sum('malicious').over(w) > 0, 1).otherwise(0)).cache()

In [ ]:
#now adjust the timediff bins
df = ts_diff(df).cache()

In [ ]:
df.printSchema()

In [ ]:
#develop quantiles for timestamp diff
bins = df.approxQuantile("timestamp_difference", [0.2, 0.4, .6, .8], 0)

In [ ]:
# create a Bucketizer instance
#bins = df.approxQuantile("timestamp_difference", [0.2, 0.4, .6, .8], 0)
#bins_2 = [float("-inf")] + bins + [float("inf")]
#bucketizer = Bucketizer(splits=bins_2, inputCol="timestamp_difference", outputCol="timestamp_bins_3")
# transform the DataFrame
#df = bucketizer.transform(df)
df_2 = df.withColumn("bin_column", when(col("timestamp_difference") <= bins[0], "1") \
              .when((col("timestamp_difference") > bins[0]) & (col('timestamp_difference') <= bins[1]), "2") \
                .when((col("timestamp_difference") > bins[1]) & (col('timestamp_difference') <= bins[2]), "3") \
                     .when((col("timestamp_difference") > bins[2]) & (col('timestamp_difference') <= bins[3]), "4") \
                     .when((col("timestamp_difference") > bins[3]), 5).otherwise('first'))
                     
              #((df["timestamp_difference"] > percentiles[0]) & (df["timestamp_difference"] <= percentiles[1]), "2")).otherwise("no"))
    #when((df["timestamp_difference"] > percentiles[1]) & (df["timestamp_difference"] <= percentiles[2]), "3"), 
    #when((df["timestamp_difference"] > percentiles[2]) & (df["timestamp_difference"] <= percentiles[3]), "4"),
    #when(df["timestamp_difference"] > percentiles[3], "5").otherwise("First"))

## bins

In [ ]:
df = bin_it(ts_diff(df))

In [ ]:
df.limit(2).toPandas()

In [ ]:
df.printSchema()

In [ ]:
type(bins)

In [ ]:
df.count()

## Now run your function and see what happens, don't forget to update for trace 3/4

In [9]:
%run trace_encode.ipynb

In [ ]:
df_23 = de_dupe()

In [ ]:
df_2315 = df_23.where((col('event_hour') == 15)).cache()

In [ ]:
df_2315.select('file_path_ext').distinct().show()

In [ ]:
df_2315.count()

In [ ]:
df_graph = create_graph3(df_2315)

In [ ]:
df_onehot = transp_expl3(df_graph).cache()

In [ ]:
df_onehot.select("Trace").distinct().count()

In [ ]:
#calculate malicious trace then check schema and how many were made 
w = Window.partitionBy("Trace")
df_onehot = df_onehot.withColumn('mal_trace', when(sum('malicious').over(w) > 0, 1).otherwise(0)).cache()
    

In [ ]:
#now develop time_diff bins
df_onehot = bin_it(df_onehot).cache()

In [ ]:
df_onehot.select("Trace").distinct().count()

In [ ]:
df_onehot.sort("Trace").limit(6).toPandas()

In [ ]:
df_onehot.printSchema()

In [ ]:
#for all columns to one hot, one hot, preserve mapping
for colm in ['file_path_ext','image_path_ext',
                                    'parent_path_ext','object','action']+["timestamp_bins"]:
    print(colm)
    df_onehot, dict_mapping, list_sparse = oneHotCol(df_onehot,colm, {}, [])

In [ ]:
df_onehot.select("Trace").distinct().count()

In [ ]:
#assemble vectors for all sparse columns - this might be enough for our ML algorithms
assembler = VectorAssembler(inputCols=list_sparse, 
                        outputCol="final_vec")
df_onehot = assembler.transform(df_onehot).cache()

list_cols = ['file_path_ext','image_path_ext',
                                    'parent_path_ext','object','action']

#turn into string
df_onehot = df_onehot.withColumn("vec2string", binary_to_string_array_udf("final_vec")).cache()
 #Generate a list of columns to drop
keep_cols = ['mal_trace','Trace','event','vec2string']
drop_cols = [col for col in df_onehot.columns  
             if col not in list_cols and col not in keep_cols]

#i want to drop any columnn not in the column list or is the malicious column
df_onehot = df_onehot.drop(*drop_cols).cache()
#first pivot aka transpose and keep all events
pivot_vec = df_onehot.groupBy('Trace').pivot('event')\
.agg(first('mal_trace'),first('vec2string')).cache()
#then consolidate the columns into a single event sequence
df_onehot = pivot_vec.select('Trace',col('e1_first(mal_trace)').alias('mal_trace'),
                          array('e1_first(vec2string)', 'e2_first(vec2string)',
                   'e3_first(vec2string)').alias('event_sequence')).cache()

In [ ]:
df_onehot.select("Trace").distinct().count()

In [ ]:
df_enc,dict_mapping = trace_encode(df_2315, 3,
                                   ['file_path_ext','image_path_ext',
                                    'parent_path_ext','object','action'])

In [ ]:
df_enc4,dict_mapping4 = trace_encode(df_2315, 4,
                                   ['file_path_ext','image_path_ext',
                                    'parent_path_ext','object','action'])

### Read/Write examples below

In [ ]:
df_enc.where((col('mal_trace')==1)&(col('malicious')==1)).count()

In [ ]:
#this math checks out 
df_enc.count()

In [ ]:
test = create_graph6(df_2315)

In [ ]:
s3_url_trusted = "s3a://sapient-bucket-trusted/"
df_enc.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/encoded/sample3")

In [ ]:
s3_url_trusted = "s3a://sapient-bucket-trusted/"
df_enc4.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/encoded/sample4")

### Create real data with just image and parent image extensions

In [ ]:
df_23 = de_dupe()

In [ ]:
df_23 = df_23.cache()

In [ ]:
df_23.count()

In [ ]:
df_23_enc_3,dict_mapping_23_3 = trace_encode(df_23, 3,
                                   ['file_path_ext','image_path_ext',
                                    'parent_path_ext','object','action'])

In [ ]:
s3_url_trusted = "s3a://sapient-bucket-trusted/"
df_23_enc_3.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/encoded/real/23Sep3")

In [ ]:
dict_mapping_23_3

In [ ]:
df_23_enc_4,dict_mapping_23_4 = trace_encode(df_23, 4,
                                   ['file_path_ext','image_path_ext',
                                    'parent_path_ext','object','action'])

In [ ]:
df_23_enc_6,dict_mapping_23_6 = trace_encode(df_23, 6,
                                   ['file_path_ext','image_path_ext',
                                    'parent_path_ext','object','action'])

## Prepare data with image path names and extensions and test with NB to observe any change in testing performance. 

In [14]:
%run trace_encode.ipynb

In [3]:
df_23 = de_dupe()

12:47PM UTC on Apr 01, 2023 --- read time: 6.950466871261597 seconds ---


In [15]:
df_23.select('image_path').distinct().show()

+--------------------+
|          image_path|
+--------------------+
|                None|
|          python.EXE|
|          python.exe|
|        Explorer.EXE|
|       taskhostw.exe|
|               other|
|            PING.EXE|
|     geckodriver.exe|
|           csrss.exe|
|         svchost.exe|
|             cmd.exe|
|         conhost.exe|
|         firefox.exe|
|    GoogleUpdate.exe|
| CompatTelRunner.exe|
|              System|
|backgroundTaskHos...|
+--------------------+



In [16]:
df_23.select('parent_image_path').distinct().show()

+-----------------+
|parent_image_path|
+-----------------+
|             None|
|            other|
|        csrss.exe|
|      svchost.exe|
|          cmd.exe|
|      conhost.exe|
| GoogleUpdate.exe|
+-----------------+



In [6]:
#run specific elements of trace encode with 23Sep data to get timestamp bins
#first create the graph. This is where I will split the data later
graph_3, tot = create_graph3(df_23)

found connections: 0.7195041179656982


event traces: 17373568


In [15]:
#now run transp_explode
start_time = time.time()

graph_3 = graph_3.cache()
trace_23_3 = transp_expl3(graph_3)
trace_23_3 = trace_23_3.cache()

print("elapsed time: " + str(time.time() - start_time))

elapsed time: 0.3261566162109375


In [10]:
#now bin this shit to get timestamp bins
start_time = time.time()

tf_bins = []
trace_23_3, tf_bins = bin_it(trace_23_3,0)
trace_23_3 = trace_23_3.cache()

print("elapsed time: " + str(time.time() - start_time))

elapsed time: 312.6835699081421


In [12]:
tf_bins

[25506.00004196167, 416052.00004577637, 2879809.000015259, 8122484.999895096]

In [25]:
#for all columns to one hot, one hot, preserve mapping -> doing this to get 23Sep3 tf bin mapping
dict_mapping = {}
list_sparse = []
df_onehot = trace_23_3

for colm in ['file_path_ext','image_path',
                                    'parent_image_path','object','action',"timestamp_bins"]:#,"pagerank_bins"]:
    #print(colm)
    df_onehot, dict_mapping, list_sparse = oneHotCol(df_onehot,colm, dict_mapping, list_sparse)

In [28]:
for k in dict_mapping.keys():
    print(dict_mapping[k])

['None', 'empty', '.dll', 'other', '.pyc', '.ini', '.pf', '.tmp', '.exe', '.bat']
['GoogleUpdate.exe', 'other', 'svchost.exe', 'python.exe', 'firefox.exe', 'Explorer.EXE', 'conhost.exe', 'None', 'System', 'cmd.exe', 'csrss.exe', 'PING.EXE', 'backgroundTaskHost.exe', 'taskhostw.exe', 'geckodriver.exe', 'python.EXE', 'CompatTelRunner.exe']
['None', 'GoogleUpdate.exe', 'other', 'svchost.exe', 'conhost.exe', 'cmd.exe', 'csrss.exe']
['PROCESS', 'FILE', 'FLOW', 'SHELL']
['OPEN', 'READ', 'MODIFY', 'INFO', 'CREATE', 'WRITE', 'DELETE', 'RENAME', 'COMMAND']
['4.0', '1.0', '0.0', '2.0', '3.0']


In [16]:
#run bin it again with pre-definied bins
start_time = time.time()

#tf_bins = []
trace_23_3 = bin_it(trace_23_3, tf_bins, 0)
trace_23_3 = trace_23_3.cache()

print("elapsed time: " + str(time.time() - start_time))

elapsed time: 0.24323320388793945


In [21]:
#confirmed both image lists so now run the 
df_23_enc_3,dict_mapping_23_3 = trace_encode(df_23, 3,
                                   ['file_path_ext','image_path',
                                    'parent_image_path','object','action'])

found connections: 0.34658265113830566


event traces: 17373568
create graph: 18.293725967407227


transposed explode and bin: 133.16281723976135


one-hot time: 322.17062401771545


total elapsed time: 345.9525034427643


In [22]:
s3_url_trusted = "s3a://sapient-bucket-trusted/"
df_23_enc_3.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/encoded/real/23Sep3_paths")

#### generate 6 event trace with pagerank and write to s3

In [2]:
%run trace_encode.ipynb

:: loading settings :: url = jar:file:/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ec2-user/.ivy2/cache
The jars for the packages stored in: /home/ec2-user/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c8db1011-b255-4c39-9569-9072f873ba5c;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 148ms :: artifacts dl 3ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	-----------------------------

23/04/01 18:16:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/01 18:16:19 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
23/04/01 18:16:20 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/04/01 18:16:20 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [21]:
df_23 = de_dupe()
#confirmed both image lists so now run the 
df_23_enc_6,dict_mapping_23_6 = trace_encode(df_23, 6,
                                   ['file_path_ext','image_path',
                                    'parent_image_path','object','action'])

 1:20PM UTC on Apr 01, 2023 --- read time: 0.4017181396484375 seconds ---
found connections: 0.8320086002349854


event traces: 14384678
create graph: 25.145016193389893
transp-explode: 25.734282970428467
transposed explode and bin: 26.262259244918823


one-hot time: 120.77667164802551


total elapsed time: 123.36127758026123


In [23]:
for k in dict_mapping_23_6.keys():
    print(dict_mapping_23_6[k])

['None', 'empty', '.dll', 'other', '.pyc', '.ini', '.tmp', '.pf', '.exe', '.bat']
['GoogleUpdate.exe', 'other', 'svchost.exe', 'csrss.exe', 'Explorer.EXE', 'python.exe', 'conhost.exe', 'firefox.exe', 'cmd.exe', 'None', 'System', 'PING.EXE', 'backgroundTaskHost.exe', 'taskhostw.exe', 'geckodriver.exe', 'python.EXE', 'CompatTelRunner.exe']
['GoogleUpdate.exe', 'other', 'None', 'svchost.exe', 'csrss.exe', 'conhost.exe', 'cmd.exe']
['PROCESS', 'FILE', 'FLOW', 'SHELL']
['OPEN', 'READ', 'MODIFY', 'INFO', 'CREATE', 'WRITE', 'DELETE', 'RENAME', 'COMMAND']
['0.0', '1.0', '3.0', '2.0', '4.0']


In [24]:
s3_url_trusted = "s3a://sapient-bucket-trusted/"
df_23_enc_6.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/encoded/real/23Sep6")

### Now automate encoding and parquet writing for 3 and 6 event traces for 24-25Sep.

In [1]:
%run read_file.ipynb
%run trace_encode.ipynb

:: loading settings :: url = jar:file:/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ec2-user/.ivy2/cache
The jars for the packages stored in: /home/ec2-user/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b5bc2923-b725-4d43-b777-00bb90be6856;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 102ms :: artifacts dl 3ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	-----------------------------

23/04/03 03:25:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/03 03:25:07 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
23/04/03 03:25:08 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/04/03 03:25:08 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
#count number of traces in 24Sep and 25 sep
df_24 = de_dupe(24)

 3:25AM UTC on Apr 03, 2023 --- read time: 4.108601331710815 seconds ---


In [3]:
df_24 = create_graph3(df_24)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/dataframe.py:148: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


found connections: 0.9773728847503662


In [4]:
df_24 = transp_expl3(df_24)
#calculate malicious trace then check schema and how many were made 
w = Window.partitionBy("Trace")
df_24 = df_24.withColumn('mal_trace', when(sum('malicious').over(w) > 0, 1).otherwise(0)).cache()
#pr_bins = []
#now develop time_diff bins, include tot for chunking 
df_24 = bin_it(df_24)

In [5]:
df_24_m = df_24.filter(col('mal_trace')==1)

In [6]:
#now down sample benign events 
#first extract those events
df_24_b = df_24.filter(col('mal_trace')==0)

In [7]:
#now filter on trace number
df_24_b = df_24_b.filter((col('Trace') <= 8235205) & 
                         (col('Trace') > 24705615))

In [8]:
#rejoin benign and malicious traces
df_24 = df_24_b.union(df_24_m).cache()

In [9]:
dict_mapping = {}
list_sparse = []
df_24, dict_mapping, list_sparse = oneHotCol(df_24,'file_path_ext', dict_mapping, list_sparse)

In [3]:
df_24.limit(2).toPandas()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,Trace,event,src,dst,timestamp,object,action,hostname,user_name,privileges,...,new_path,file_path,direction,logon_id,requesting_domain,requesting_user,malicious,mal_trace,timestamp_bins,file_path_ext_sparse
0,1240,e2,73d89b62-3424-41ea-9cb9-f2219035997d,59cfbe40-9d95-4ebc-a713-16d1c78a46d5,2019-09-24 20:21:29.083,PROCESS,OPEN,SysClient0419.systemia.com,None,None,...,None,None,None,None,None,None,0,0,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,2404,e1,bcd4a5c8-7f1b-4809-81db-ce4b4a9e6890,73d89b62-3424-41ea-9cb9-f2219035997d,2019-09-24 14:09:27.523,FILE,MODIFY,SysClient0419.systemia.com,None,None,...,None,\Device\HarddiskVolume1\Windows\System32\WDI\{...,None,None,None,None,0,0,NaN,"(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [10]:
for col in ['image_path','parent_image_path']:
    df_24, dict_mapping, list_sparse = oneHotCol(df_24,col, dict_mapping, list_sparse)

In [1]:
df_24.limit(2).toPandas()

NameError: name 'df_24' is not defined

In [11]:
df_24, dict_mapping, list_sparse = oneHotCol(df_24,'object', dict_mapping, list_sparse)

In [12]:
df_24, dict_mapping, list_sparse = oneHotCol(df_24,'action', dict_mapping, list_sparse)

In [13]:
df_24, dict_mapping, list_sparse = oneHotCol(df_24,'timestamp_bins', dict_mapping, list_sparse)

In [14]:
#assemble vectors for all sparse columns - this might be enough for our ML algorithms
assembler = VectorAssembler(inputCols=list_sparse, 
                        outputCol="final_vec")

df_24 = assembler.transform(df_24).cache()

#turn into string
df_24 = df_24.withColumn("vec2string", binary_to_string_array_udf("final_vec")).cache()

In [17]:
list_sparse

['file_path_ext_sparse',
 'image_path_sparse',
 'parent_image_path_sparse',
 'object_sparse',
 'action_sparse',
 'timestamp_bins_sparse']

In [15]:
list_cols = ['file_path_ext','image_path',
             'parent_image_path','object',
             'action','timestamp_bins']

In [17]:
df_24.limit(2).toPandas()

,Trace,event,mal_trace,vec2string
0,158106,e1,1,010000000100000000010000000010001000000000001
1,158106,e2,1,100000000001000000000100000100010000000001000


In [ ]:
df_24.printSchema

root
 |-- Trace: long (nullable = false)
 |-- event: string (nullable = false)
 |-- mal_trace: integer (nullable = false)
 |-- vec2string: string (nullable = true)



In [16]:
keep_cols = ['mal_trace','Trace','event','vec2string']
drop_cols = [col for col in df_24.columns  
             if col not in list_cols and col not in keep_cols]

#i want to drop any columnn not in the column list or is the malicious column
df_24 = df_24.drop(*drop_cols).cache()

In [17]:
from pyspark.sql.functions import *
#filter df by traces again
df_24_1 = df_24.filter(col('Trace') <= 8235205) #df_24.where(col("Trace") <= 8235205) 
df_24_2 = df_24.filter((col('Trace') > 24705615) | 
                      (col("mal_trace") == 1)) #df_24.where(col("Trace") > 24705615) 

In [18]:
#first pivot aka transpose and keep all events
df_24_1 = df_24_1.groupBy('Trace').pivot('event')\
.agg(first('mal_trace'),first('vec2string')).cache()
#then consolidate the columns into a single event sequence
df_24_1 = df_24_1.select('Trace',col('e1_first(mal_trace)').alias('mal_trace'),
                          array('e1_first(vec2string)', 'e2_first(vec2string)',
                   'e3_first(vec2string)').alias('event_sequence')).cache()

#first pivot aka transpose and keep all events
df_24_2 = df_24_2.groupBy('Trace').pivot('event')\
.agg(first('mal_trace'),first('vec2string')).cache()
#then consolidate the columns into a single event sequence
df_24_2 = df_24_2.select('Trace',col('e1_first(mal_trace)').alias('mal_trace'),
                          array('e1_first(vec2string)', 'e2_first(vec2string)',
                   'e3_first(vec2string)').alias('event_sequence')).cache()

df_24 = df_24_1.union(df_24_2)

df_24_1.unpersist()
df_24_2.unpersist()

DataFrame[Trace: bigint, mal_trace: int, event_sequence: array<string>]

In [19]:
df_24 = df_24.cache()

In [24]:
df_24.limit(2).toPandas()

,Trace,mal_trace,event_sequence
0,158106,1,[010000000100000000010000000010001000000000001...
1,167219,1,[100000000100000000010000000001000010000000001...


In [20]:
for k in dict_mapping.keys():
    print(dict_mapping[k])

['None', 'empty', '.dll', 'other', '.ini', '.exe', '.pf', '.tmp']
['other', 'GoogleUpdate.exe', 'svchost.exe', 'csrss.exe', 'conhost.exe', 'cmd.exe', 'taskhostw.exe', 'PING.EXE', 'Explorer.EXE']
['None', 'GoogleUpdate.exe', 'svchost.exe', 'other', 'csrss.exe', 'conhost.exe', 'cmd.exe']
['PROCESS', 'FILE', 'SHELL']
['OPEN', 'READ', 'CREATE', 'COMMAND', 'MODIFY', 'WRITE', 'DELETE']
['4.0', '3.0', '2.0', '0.0', '1.0']


In [21]:
#now write this data to parquet
write_after(df_24,24,3,20,"actual")

split time: 1.5885505676269531
write: part 1


part 1 write time: 209.6388964653015
write: part 2


part 2 write time: 423.24907088279724
write: part 3


part 3 write time: 625.9810876846313
write: part 4


part 4 write time: 822.8803811073303
write: part 5


...

part 16 write time: 3322.5822825431824
write: part 17


part 17 write time: 3531.944423675537
write: part 18


part 18 write time: 3737.7635962963104
write: part 19


part 19 write time: 3949.1249299049377
write: part 20


part 20 write time: 4162.245982885361
total write time: 4162.246063947678


### Now create the df for 24Sep trace 6

In [1]:
%run read_file.ipynb
%run trace_encode.ipynb

:: loading settings :: url = jar:file:/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ec2-user/.ivy2/cache
The jars for the packages stored in: /home/ec2-user/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a780743d-4200-4ebc-911e-75d3c137c869;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 138ms :: artifacts dl 5ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	-----------------------------

23/04/03 08:33:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/03 08:33:15 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
23/04/03 08:33:16 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/04/03 08:33:16 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
df_24 = de_dupe(24)

 8:33AM UTC on Apr 03, 2023 --- read time: 4.531464576721191 seconds ---


In [3]:
df_24 = create_graph6(df_24)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/dataframe.py:148: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


found connections: 1.666179895401001


In [4]:
#TODO: use this snippet to write graph df into S3
s3_url_trusted = "s3a://sapient-bucket-trusted/"
df_24.write.option("maxRecordsPerFile", 300000).mode("overwrite")\
        .parquet(f'''{s3_url_trusted}/prod/graph/encoded/real/
                    unprocessed/24Sep6''')

ERROR:root:KeyboardInterrupt while sending command.              (0 + 32) / 200]
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [4]:
df_24 = transp_expl(df_24)
#calculate malicious trace then check schema and how many were made 
w = Window.partitionBy("Trace")
df_24 = df_24.withColumn('mal_trace', when(sum('malicious').over(w) > 0, 1).otherwise(0)).cache()
#now develop time_diff bins, include tot for chunking 
df_24 = bin_it(df_24)

In [5]:
df_24_m = df_24.filter(col('mal_trace')==1)
df_24_b = df_24.filter(col('mal_trace')==0)

In [6]:
#now filter on trace number
df_24_b = df_24_b.filter((col('Trace') > 8235205) & 
                         (col('Trace') <= 12352808) & 
                         (col('Trace') > 24705615)& 
                         (col('Trace') <= 28823217))
#rejoin benign and malicious traces
df_24 = df_24_b.union(df_24_m).cache()

In [ ]:
dict_mapping = {}
list_sparse = []
start_time = time.time()
df_24, dict_mapping, list_sparse = oneHotCol(df_24,'file_path_ext', dict_mapping, list_sparse)
print("elapsed time: " + str(time.time() - start_time))

In [ ]:
start_time = time.time()
for col in ['image_path','parent_image_path']:
    df_24, dict_mapping, list_sparse = oneHotCol(df_24,col, dict_mapping, list_sparse)
print("elapsed time: " + str(time.time() - start_time))

In [ ]:
start_time = time.time()
df_24, dict_mapping, list_sparse = oneHotCol(df_24,'object', dict_mapping, list_sparse)
print("elapsed time: " + str(time.time() - start_time))

In [ ]:
start_time = time.time()
df_24, dict_mapping, list_sparse = oneHotCol(df_24,'action', dict_mapping, list_sparse)
print("elapsed time: " + str(time.time() - start_time))

In [ ]:
start_time = time.time()
df_24, dict_mapping, list_sparse = oneHotCol(df_24,'timestamp_bins', dict_mapping, list_sparse)
print("elapsed time: " + str(time.time() - start_time))

In [ ]:
#assemble vectors for all sparse columns - this might be enough for our ML algorithms
assembler = VectorAssembler(inputCols=list_sparse, 
                        outputCol="final_vec")

df_24 = assembler.transform(df_24).cache()

#turn into string
df_24 = df_24.withColumn("vec2string", binary_to_string_array_udf("final_vec")).cache()

In [ ]:
list_cols = ['file_path_ext','image_path',
             'parent_image_path','object',
             'action','timestamp_bins']

In [ ]:
keep_cols = ['mal_trace','Trace','event','vec2string']
drop_cols = [col for col in df_24.columns  
             if col not in list_cols and col not in keep_cols]

#i want to drop any columnn not in the column list or is the malicious column
df_24 = df_24.drop(*drop_cols).cache()

In [ ]:
from pyspark.sql.functions import *
#filter df by traces again
df_24_1 = df_24.filter((col('Trace') > 8235205) & 
                         (col('Trace') <= 12352808)) 
df_24_2 = df_24.filter(((col('Trace') > 24705615) & 
                         (col('Trace') <= 28823217))| 
                      (col("mal_trace") == 1))

In [ ]:
#first pivot aka transpose and keep all events
df_24_1 = df_24_1.groupBy('Trace').pivot('event')\
.agg(first('mal_trace'),first('vec2string')).cache()
#then consolidate the columns into a single event sequence
df_24_1 = df_24_1.select('Trace',col('e1_first(mal_trace)').alias('mal_trace'),
                          array('e1_first(vec2string)', 'e2_first(vec2string)',
                   'e3_first(vec2string)').alias('event_sequence')).cache()

#first pivot aka transpose and keep all events
df_24_2 = df_24_2.groupBy('Trace').pivot('event')\
.agg(first('mal_trace'),first('vec2string')).cache()
#then consolidate the columns into a single event sequence
df_24_2 = df_24_2.select('Trace',col('e1_first(mal_trace)').alias('mal_trace'),
                          array('e1_first(vec2string)', 'e2_first(vec2string)',
                   'e3_first(vec2string)').alias('event_sequence')).cache()

df_24 = df_24_1.union(df_24_2).cache()

df_24_1.unpersist()
df_24_2.unpersist()

In [ ]:
for k in dict_mapping.keys():
    print(dict_mapping[k])

In [ ]:
#now write this data to parquet
write_after(df_24,24,6,20,"actual")

In [7]:
#create a dictionary to preserve mappings for everything
mapping = {}
start_time = time.time()
for d in [24,25]:
    
    print(f"  processing {d} Sep")
    #create dataframe before breaking it down into traces
    df = de_dupe(d)
    
    for trace in [3,6]: 
        
        print(f"    processing {trace} event traces")
        
        #encoding
        df_enc, dict_mapping = trace_encode(df, trace,
                                       ['file_path_ext','image_path',
                                        'parent_image_path','object','action'])
        
        print(f"    {d} Sep-{trace} encoded")
        print("    encoding time: "+str(time.time()-start_time))
        
        #preserve mapping
        mapping[f"{d}-{trace}"] = dict_mapping
    
    print(f"  {day} Sep processing time: "+str(time.time()-start_time))
    
print("total time: "+str(time.time()-start_time))

  processing 24 Sep


 3:33PM UTC on Apr 01, 2023 --- read time: 136.55989837646484 seconds ---


    processing 3 event traces


found connections: 0.6804215908050537
create graph: 0.718543529510498
transp-explode: 1.1523950099945068
bin time: 1.6936383247375488


one-hot time: 1225.5698642730713


23/04/01 15:56:06 ERROR Executor: Exception in task 27.0 in stage 236.0 (TID 4537)
java.lang.OutOfMemoryError: GC overhead limit exceeded
	at java.lang.reflect.Array.newInstance(Array.java:75)
	at java.io.ObjectInputStream.readArray(ObjectInputStream.java:2082)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1656)
	at java.io.ObjectInputStream.readArray(ObjectInputStream.java:2118)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1656)
	at java.io.ObjectInputStream.defaultReadFields(ObjectInputStream.java:2430)
	at java.io.ObjectInputStream.readSerialData(ObjectInputStream.java:2354)
	at java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2212)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1668)
	at java.io.ObjectInputStream.readObject(ObjectInputStream.java:502)
	at java.io.ObjectInputStream.readObject(ObjectInputStream.java:460)
	at org.apache.spark.serializer.JavaDeserializationStream.readObject(JavaSerializer.scala

23/04/01 15:56:13 ERROR TaskSchedulerImpl: Exception in statusUpdate
java.util.concurrent.RejectedExecutionException: Task org.apache.spark.scheduler.TaskResultGetter$$Lambda$6525/192258468@23c57067 rejected from java.util.concurrent.ThreadPoolExecutor@67deacaf[Terminated, pool size = 0, active threads = 0, queued tasks = 0, completed tasks = 4512]
	at java.util.concurrent.ThreadPoolExecutor$AbortPolicy.rejectedExecution(ThreadPoolExecutor.java:2063)
	at java.util.concurrent.ThreadPoolExecutor.reject(ThreadPoolExecutor.java:830)
	at java.util.concurrent.ThreadPoolExecutor.execute(ThreadPoolExecutor.java:1379)
	at org.apache.spark.scheduler.TaskResultGetter.enqueueFailedTask(TaskResultGetter.scala:137)
	at org.apache.spark.scheduler.TaskSchedulerImpl.liftedTree2$1(TaskSchedulerImpl.scala:821)
	at org.apache.spark.scheduler.TaskSchedulerImpl.statusUpdate(TaskSchedulerImpl.scala:794)
	at org.apache.spark.scheduler.local.LocalEndpoint$$anonfun$receive$1.applyOrElse(LocalSchedulerBackend.sc

Py4JJavaError: An error occurred while calling o1432.pivot.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 27 in stage 236.0 failed 1 times, most recent failure: Lost task 27.0 in stage 236.0 (TID 4537) (ip-172-16-26-180.us-west-2.compute.internal executor driver): java.lang.OutOfMemoryError: GC overhead limit exceeded
	at java.lang.reflect.Array.newInstance(Array.java:75)
	at java.io.ObjectInputStream.readArray(ObjectInputStream.java:2082)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1656)
	at java.io.ObjectInputStream.readArray(ObjectInputStream.java:2118)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1656)
	at java.io.ObjectInputStream.defaultReadFields(ObjectInputStream.java:2430)
	at java.io.ObjectInputStream.readSerialData(ObjectInputStream.java:2354)
	at java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2212)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1668)
	at java.io.Objec

### Data processing the right way

In [1]:
%run read_file.ipynb


:: loading settings :: url = jar:file:/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ec2-user/.ivy2/cache
The jars for the packages stored in: /home/ec2-user/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f16eb368-7fb5-498a-b7be-3caef9613bd4;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
downloading https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar ...
	[SUCCESSFUL ] graphframes#graphframes;0.8.2-spark3.2-s_2.12!graphframes.jar (46ms)
downloading https://repo1.maven.org/maven2/org/slf4j/slf4j-api/1.7.16/slf4j-api-1.7.16.jar ...
	[SUCCESSFUL ] org.slf4j#slf4j-api;1.7.16!slf4j-api.jar (33ms)
:: resolution report :: resolve 804ms :: artifacts dl 81ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	--------

23/04/04 01:08:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/04 01:08:22 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [3]:
%run trace_encode.ipynb

In [4]:
df = de_dupe()

 1:09AM UTC on Apr 04, 2023 --- read time: 6.179586887359619 seconds ---


In [ ]:
#write using the write after function
write_after(df_enc, day, trace)
print(f"    {d}-{trace} written")
print("    write time: "+str(time.time()-start_time))